<a href="https://colab.research.google.com/github/vivekvision/PySparkDataWrangling/blob/main/GenerateMissingDatesFillValues2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generate missing data and fill values - Generate each day 

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 51.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=8c12f06860d4ae7d8462ecaac490168f6c8151f103bc89f185d8f2f7896e3b1d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pandas as pd
import pyspark.sql.functions as sf

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("testApp")\
        .getOrCreate()

In [3]:
from pyspark.sql import Window

In [35]:
# construct a data-frame 
data = [(123, 1, "01/01/2021",),
        (123, 0, "01/02/2021",),
        (123, 1, "01/03/2021",),
        (123, 0, "01/06/2021",),
        (123, 0, "01/08/2021",),
        (777, 0, "01/01/2021",),
        (777, 1, "01/03/2021",), ]

df = spark.createDataFrame(data, ("ID", "FLAG", "DATE",)).withColumn("DATE", sf.to_date(sf.col("DATE"), "dd/MM/yyyy"))

In [36]:
window_spec = Window.partitionBy("ID").orderBy("DATE")


# window function lead return row located after offset rows from current row : In this case offset rows is 1 
# expr() is the SQL function to execute SQL-like expressions and use an existing DataFrame column value as the expression argument

# sequence function to generate all intermedicate dates
df = df.withColumn("DatesBetween", sf.sequence(sf.col("DATE"), sf.lead("DATE", 1).over(window_spec), sf.expr("interval 1 day")))


In [37]:
df.toPandas()

,ID,FLAG,DATE,DatesBetween
0,123,1,2021-01-01,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
1,123,0,2021-02-01,"[2021-02-01, 2021-02-02, 2021-02-03, 2021-02-0..."
2,123,1,2021-03-01,"[2021-03-01, 2021-03-02, 2021-03-03, 2021-03-0..."
3,123,0,2021-06-01,"[2021-06-01, 2021-06-02, 2021-06-03, 2021-06-0..."
4,123,0,2021-08-01,None
5,777,0,2021-01-01,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
6,777,1,2021-03-01,None


In [38]:
# explode function is used to explode list column value to rows 
df = df.withColumn("DATE", sf.explode("DatesBetween"))

df.toPandas()

,ID,FLAG,DATE,DatesBetween
0,123,1,2021-01-01,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
1,123,1,2021-01-02,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
2,123,1,2021-01-03,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
3,123,1,2021-01-04,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
4,123,1,2021-01-05,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
...,...,...,...,...
271,777,0,2021-02-25,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
272,777,0,2021-02-26,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
273,777,0,2021-02-27,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
274,777,0,2021-02-28,"[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-0..."
